In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "~/git/swiftai")' SwiftAI

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
//export
import Path
import SwiftAI
import TensorFlow

In [ ]:
public typealias TF = Tensor<Float>
public typealias TI = Tensor<Int32>

In [ ]:
//export
import Python

let gym = Python.import("gym")
let np = Python.import("numpy")

In [ ]:
struct Observation {
    public let obs: TF
}

let empty_obs = Observation(obs: TF(zeros: [210, 160, 3]))

struct Action {
    public let action: Int
}

struct Reward : AdditiveArithmetic {
    public let r: Float
    
}
let zero_reward = Reward(r:0)

// Define how to add reward objects
func +(lhs: Reward, rhs: Reward) -> Reward {
        let r = lhs.r + rhs.r
        return Reward(r:r)
}

# Need to deal with the Gym environment

In [ ]:
func np_to_tf(_ arr: PythonObject) -> TF {
    return Tensor<Float> ( numpy: np.array(arr, dtype: np.float32))!
}

In [ ]:
func np_to_obs(_ arr: PythonObject) -> Observation {
    let obs = Tensor<Float> ( numpy: np.array(arr, dtype: np.float32))
    if obs == nil {
        print("np_to_obs: Oh no! observation did not parse!")
    }
    return Observation(obs: obs!)
}

In [ ]:
func reset_env(_ env: PythonObject) -> Observation {
    return np_to_obs(env.reset())
}

func step_env(env: PythonObject, act: Action) -> (Observation, Reward, Bool) {
    let res = env.step(act.action)
    let obs = np_to_obs(res[0])
    let r = Float(res[1])!
    let cont = Bool(res[2])!
    
    return (obs, Reward(r:r), cont )
}

In [ ]:
// steps n times, summing reward over steps.
func step_n_times(n: Int = 4, env: PythonObject, act: Action) ->
    (Observation, Reward, Bool)
{
    var rew = Reward(r: 0)

    var res = env.step(act.action)
    rew = rew + Reward(r: Float(res[1])! )
    
    let range: ClosedRange<Int> = 1...n
    for _ in range {
        res = env.step(act.action)
        rew = rew + Reward(r: Float(res[1])! )
    }
    
    let obs = np_to_obs(res[0])
    let running  = Bool(res[2])!
    
    return (obs, rew, running)
}

# Now try to build a DQN

In [ ]:
public struct DQNModel: Layer {
    public var conv1: ConvLayer
    public var conv2: ConvLayer
    public var conv3: ConvLayer
    public var pool = GlobalAvgPool2D<Float>()
    public var linear1: Dense<Float>
    public var linear2: Dense<Float>

    
    public init(nActions: Int){
        conv1 = ConvLayer(3, 32, ks: 8)
        conv2 = ConvLayer(32, 64, ks: 4, stride: 2)
        conv3 = ConvLayer(64, 64, ks: 3)
        linear1 = Dense(inputSize: 64, outputSize: 256, activation: relu) 
        linear2 = Dense(inputSize: 256, outputSize: nActions) 
    }
    
    @differentiable
    public func callAsFunction(_ input: TF) -> TF {
       return input.compose(conv1, conv2, conv3, pool, linear1, linear2)
        
    }
}

In [ ]:
var p1 = DQNModel(nActions: 6)

In [ ]:
var vars = p1.variables

In [ ]:
p1.variables = vars

In [ ]:
var  env = gym.make("PongNoFrameskip-v4")
var s1 = env.reset().reshape([1, 210,160,3])

In [ ]:
// how many actions are there... 
env.action_space

In [ ]:
var (a,b,c) = step_n_times(n:10, env: env, act: Action(action:1))

In [ ]:
(a.obs - np_to_tf(s1)).sum()

In [ ]:
func select_greedy_action(s: SA, 
                          obs: Observation) -> Action {
    
    let (_, a) = best_action_value(s:s, cs:obs)
    
    return a
}

func best_action_value(s: SA, 
                       cs: Observation) -> (Reward, Action)
{
    var best_value: Float = 0
    var best_action = Action(action:0)
    let r : Range = 0..<4
    for i in r {
        var a = Action(action:i)
        var pair  = StateActionPair(state: cs, act:a) 
        if ( (s[pair] ?? 0) > best_value) {
            best_value = s[pair] ?? 0
            best_action = a
        }
    }
    
    return (Reward(reward:best_value), 
            best_action)
}

In [ ]:
func select_eps_greedy_action(s: SA, 
                              obs: Observation, 
                              n_actions: Int, 
                              eps: Float) -> Action {
    
    let (val, act) = best_action_value(s:s, cs: obs)
    
    let r = Float( random.random() )!
    
    if (r < eps) {
        let r_act = Int(random.randint(0, n_actions))!
        return Action(action: r_act)
        
    } else {
        return act
    }
}

func q_learning(sr:inout SA, current_obs: Observation, 
                next_obs: Observation, r: Reward, a: Action, 
                g: Float, lr: Float) -> SA
{
    let (best_value, _) = best_action_value(s:sr, 
                                             cs: next_obs)
    
    let Q_target = r.reward + g * best_value.reward
    let pair = StateActionPair(state: current_obs, act:a)
    let Q_error = Q_target - (sr[pair] ?? 0)
    
    
    sr[pair] = (sr[pair] ?? 0) + lr * Q_error
    
    return sr
}

In [ ]:
let Gamma : Float = 0.95

var epsilon: Float = 1.0
let EPS_DECAY_RATE : Float = 0.99939
let LEARNING_RATE : Float = 0.8

let TEST_EPISODES = 100
let MAX_GAMES = 150001

In [ ]:
games_count = 0
epsilon = 1.0

var total_reward : Float = 0

while (games_count < MAX_GAMES) {
    let act = select_eps_greedy_action(s: sr, obs:obs, 
                                       n_actions:n_actions,
                                       eps: epsilon
                                      )
    
    let (next_obs, reward, done) = step_env(env: env, act:act)
    sr = q_learning(sr: &sr, current_obs: obs, next_obs:next_obs, 
                   r: reward, a: act, g: Gamma,
                   lr: LEARNING_RATE)
    obs = next_obs
    total_reward += reward.reward
    //print(total_reward)
    if (done) {
        epsilon *= EPS_DECAY_RATE
    
        if ((games_count % 1000) == 0) {
            let test_reward = test_game(env: env, s: sr, no_games: TEST_EPISODES)
            test_rewards_list.append(test_reward)
            print("Games count: " + String(games_count) + " Epsilon: " + String(epsilon))
            print(test_reward.reward)
        }
        
        // do testing logic... 
        obs = reset_env(env: env)
        games_count+=1
        total_reward=0
    }
}

In [ ]:
p1.shape

In [ ]:
//reset_env(env)

In [ ]:
//step_n_times(n: 3, env: env, act: Action(action:1))